In [ ]:
%%javascript
$('#appmode-leave').hide();                          // Hides the edit app button.
$('#appmode-busy').hide();                           // Hides the kernel busy indicator.

In [ ]:
# Google Forms / Sheets Code
import urllib.request
from bs4 import BeautifulSoup
import requests, warnings
def get_questions(in_url):
    res = urllib.request.urlopen(in_url)
    soup = BeautifulSoup(res.read(), 'html.parser')
    get_names = lambda f: [v for k,v in f.attrs.items() if 'label' in k]
    get_name = lambda f: get_names(f)[0] if len(get_names(f))>0 else 'unknown'
    all_questions = soup.form.findChildren(attrs={'name': lambda x: x and x.startswith('entry.')})
    return {get_name(q): q['name'] for q in all_questions}
def submit_response(form_url, cur_questions, verbose=False, **answers):
    submit_url = form_url.replace('/viewform', '/formResponse')
    form_data = {'draftResponse':[],
                'pageHistory':0}
    for v in cur_questions.values():
        form_data[v] = ''
    for k, v in answers.items():
        if k in cur_questions:
            form_data[cur_questions[k]] = v
        else:
            warnings.warn('Unknown Question: {}'.format(k), RuntimeWarning)
    if verbose:
        print(form_data)
    user_agent = {'Referer':form_url,
                  'User-Agent': "Mozilla/5.0 (X11; Linux i686) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/28.0.1500.52 Safari/537.36"}
    return requests.post(submit_url, data=form_data, headers=user_agent)

In [ ]:
import os
import ipywidgets as ipw
from glob import glob
import json
import pandas as pd
import numpy as np
from PIL import Image
from itertools import cycle
from io import BytesIO
from time import time
USERNAME=os.environ.get('APPMODE_USER', 'Not logged in')
HOSTNAME=os.environ.get('HOSTNAME', 'anon')
BUTTONS_PER_ROW = 3
BUTTON_WIDTH = "150px"
UNKNOWN_OPTION = 'Unknown'

In [ ]:
with open('task.json', 'r') as f:
    annotation_task = json.load(f)
    annot_df = pd.DataFrame(annotation_task['dataset']['dataframe'])
print('Loaded annotation task', annot_df.shape[0], 'images')
button_ids = annot_df[annotation_task['dataset']['output_labels']].unique().tolist()
if UNKNOWN_OPTION is not None:
    button_ids += [UNKNOWN_OPTION]

image_path_dict = {c_path: os.path.join(annotation_task['dataset']['base_image_directory'], 
                                    c_path)
                                    for c_path in annot_df[annotation_task['dataset']['image_path']]}
image_keys = list(image_path_dict.keys())

In [ ]:
FORM_URL = annotation_task['google_forms']['form_url']
anno_questions = get_questions(FORM_URL)

In [ ]:
global out_rows
out_rows = []
def submit_label(label_id, label_name):
    """
    should be linked to some kind of database
    """
    global out_rows
    print('Submitting', label_name, 'for', label_id, 'from', USERNAME, 'part of', HOSTNAME, '@', time())
    submit_response(FORM_URL, anno_questions, 
                    annotator=USERNAME, 
                    session=HOSTNAME, 
                    time = time(),
                    item_id = label_id,
                    label = label_name,
                    task = 'pneumonia_annotation'
                   )
    out_rows += [dict(label_name=label_name, label_id=label_id, user=USERNAME, session=HOSTNAME, time=time())]
    return pd.DataFrame(out_rows).to_html()

In [ ]:
title_box = ipw.HTML(value=f'<h1> Welcome {USERNAME}</h1><h2> Select the most appropriate label for the given image')
cur_image_view = ipw.Image(layout=ipw.Layout(width="512px"),  
                  disabled=True)
cur_img_id = ipw.Text(layout=ipw.Layout(width="64px"),  
                  disabled=True)
annot_results = ipw.HTML(value='')

SESSION_ID = None
np.random.shuffle(image_keys)
image_iter = cycle(iter(image_keys))

def get_next_image():
    c_key = next(image_iter)
    cur_img_id.value = c_key
    c_img = Image.open(image_path_dict[c_key])
    bio_obj = BytesIO()
    c_img.save(bio_obj, format='png')
    bio_obj.seek(0)
    cur_image_view.value=bio_obj.read()

def on_click(btn):
    if btn is not None:
        annot_results.value = submit_label(cur_img_id.value, btn.description)
    get_next_image()
        

def mk_btn(description):
    btn = ipw.Button(description=description, layout=ipw.Layout(width=BUTTON_WIDTH))
    btn.on_click(on_click)
    return btn

rows = []
c_row = []
for i, but_name in enumerate(button_ids, 1):
    c_row+=[mk_btn(but_name)]
    if (i % BUTTONS_PER_ROW)==0:
        rows+=[ipw.HBox(c_row)]
        c_row=[]
rows+=[ipw.HBox(c_row)]
on_click(None) # initialize
ipw.VBox((title_box, cur_image_view)+tuple(rows)+(annot_results,))